In [1]:
# in case there are any problems with importing because path is wrong
import sys
sys.path.append('/Users/daniel/Princeton Dropbox/Daniel Gurevich/Research/discrete_sr/code/SPIDER_discrete')

In [2]:
import h5py
import numpy as np
from utils import save, load
from library import *
from continuous.process_library_terms import *

def load_matlab_v73(mat_file_path):
    """
    Loads MATLAB v7.3 .mat data using h5py and converts it to Python-readable formats.

    Parameters:
    - mat_file_path (str): Path to the .mat file.

    Returns:
    - dict: A dictionary with MATLAB variable names as keys and corresponding NumPy arrays as values.
    """
    try:
        # Open the HDF5 file
        with h5py.File(mat_file_path, 'r') as f:
            mat_data = {}

            def recursively_load(group):
                """
                Recursively load MATLAB v7.3 groups into dictionaries.
                """
                data = {}
                for key, item in group.items():
                    if isinstance(item, h5py.Dataset):
                        data[key] = np.array(item)  # Convert HDF5 dataset to NumPy array
                    elif isinstance(item, h5py.Group):
                        data[key] = recursively_load(item)  # Recursively process groups
                return data

            # Load all variables from the root group
            mat_data = recursively_load(f)

        return mat_data
    except Exception as e:
        print(f"Error loading .mat file: {e}")
        return None

# Path to your MATLAB v7.3 .mat file
mat_file_path = "Matteo.mat"  # Update "data.mat" to your fil  # Replace with your .mat file path

# Load the .mat file
python_data = load_matlab_v73(mat_file_path)

# Display the loaded data
if python_data:
    for var_name, data in python_data.items():
        if isinstance(data, np.ndarray):
            print(f"Variable: {var_name}, Shape: {data.shape}, Type: {type(data)}")
        else:
            print(f"Variable: {var_name}, Type: {type(data)} (nested structure)")

s = python_data['s']  # Replace 's' with the actual key name if it's different

# Extract the first layer (V) and the second layer (U)
V = s[:, 0, :, :]  # First layer
U = s[:, 1, :, :]  # Second layer
Lx = 2*np.pi; Ly = 2*np.pi; Lt = 0.01;
Nx = 2048; Ny = 2048; Nt = 100 # analytical
dx = Lx/Nx; dy = Ly/Ny; dt = Lt/Nt;

# Display their shapes

U = np.transpose(U, (2,1,0))
V = np.transpose(V, (2,1,0))

def pressure_poisson(U, V, dx, dy, density=1.0):
    nx, ny, nt = U.shape
    kx = np.fft.fftfreq(nx, d=dx) * 2 * np.pi
    ky = np.fft.rfftfreq(ny, d=dy) * 2 * np.pi
    kx, ky = np.meshgrid(kx, ky, indexing='ij')
    k_squared = kx**2 + ky**2
    k_squared[0, 0] = np.inf 
    P = np.zeros((nx, ny, nt))
    for t in range(nt):
        u_FT = np.fft.rfftn(U[:, :, t])
        v_FT = np.fft.rfftn(V[:, :, t])
        i = 1j
        dxu = np.fft.irfftn(i * kx * u_FT, s=(nx, ny))
        dyu = np.fft.irfftn(i * ky * u_FT, s=(nx, ny))
        dxv = np.fft.irfftn(i * kx * v_FT, s=(nx, ny))
        dyv = np.fft.irfftn(i * ky * v_FT, s=(nx, ny))
        rhs = dxu**2 + 2 * dyu * dxv + dyv**2
        rhs_FT = np.fft.rfftn(rhs)
        pressure_FT = density * rhs_FT / k_squared
        pressure = np.fft.irfftn(pressure_FT, s=(nx, ny))
        P[:, :, t] = pressure
    return P

P = pressure_poisson(U, V, dx, dy)

#SAMPLE # DOWNSAMPLING
#xsample = 2
#ysample = xsample
#tsample = 1

#U = U[::xsample, ::ysample, ::tsample]
#V = V[::xsample, ::ysample, ::tsample]
#P = P[::xsample, ::ysample, ::tsample]

#Nx = 2048/xsample; Ny = 2048/ysample; Nt = 100/tsample
#dx = Lx/Nx; dy = Ly/Ny; dt = Lt/Nt;

print(f"V: Shape = {V.shape}, Type = {type(V)}")
print(f"U: Shape = {U.shape}, Type = {type(U)}")
print(f"P: Shape = {P.shape}, Type = {type(P)}")

Variable: s, Shape: (100, 2, 2048, 2048), Type: <class 'numpy.ndarray'>


/var/folders/p_/dc9h9qcj62qgv461rz72zx_80000gn/T/ipykernel_37904/815513373.py:82: DeprecationWarning: `axes` should not be `None` if `s` is not `None` (Deprecated in NumPy 2.0). In a future version of NumPy, this will raise an error and `s[i]` will correspond to the size along the transformed axis specified by `axes[i]`. To retain current behaviour, pass a sequence [0, ..., k-1] to `axes` for an array of dimension k.
  dxu = np.fft.irfftn(i * kx * u_FT, s=(nx, ny))
/var/folders/p_/dc9h9qcj62qgv461rz72zx_80000gn/T/ipykernel_37904/815513373.py:83: DeprecationWarning: `axes` should not be `None` if `s` is not `None` (Deprecated in NumPy 2.0). In a future version of NumPy, this will raise an error and `s[i]` will correspond to the size along the transformed axis specified by `axes[i]`. To retain current behaviour, pass a sequence [0, ..., k-1] to `axes` for an array of dimension k.
  dyu = np.fft.irfftn(i * ky * u_FT, s=(nx, ny))
/var/folders/p_/dc9h9qcj62qgv461rz72zx_80000gn/T/ipykernel_3

V: Shape = (2048, 2048, 100), Type = <class 'numpy.ndarray'>
U: Shape = (2048, 2048, 100), Type = <class 'numpy.ndarray'>
P: Shape = (2048, 2048, 100), Type = <class 'numpy.ndarray'>


In [3]:
import numpy as np

def fourier_filter(field, width, filterscale, kernel):
    """
    Filters a given field based on the specified mode and kernel.

    Parameters:
    - field: the field to base the filter on
    - width: controls the filter's boundary steepness
    - filterscale: determines the filtering scale
    - kernel: "tanh", "gaussian", "heaviside", etc.
    
    Returns:
    - filtered_field: the filtered field
    """
    # Generate the grid

    hey = np.linspace(0,Nx//2,(Nx//2)+1)
    hey2 = np.linspace(-((Nx//2)-1),-1,((Nx//2)-1))
    q = np.concatenate((hey,hey2))

    X = np.outer(q, np.ones_like(q))
    Y = np.outer(np.ones_like(q), q)
    R = np.sqrt(X**2 + Y**2)
    
    # Create the filter kernel
    if kernel == "tanh":
        G = -0.5 * (np.tanh(width * (R - filterscale)) - 1)
    elif kernel == "gaussian":
        G = np.exp(-((R - filterscale) ** 2) / (2 * width**2))
    elif kernel == "Itanh":
        G = -0.5 * (-np.tanh(width * (R - filterscale)) - 1)
    elif kernel == "Igaussian":
        G = 1 - np.exp(-((R - filterscale) ** 2) / (2 * width**2))
    elif kernel == "symheaviside":
        G = (np.abs(R) <= filterscale).astype(float)
    elif kernel == "Isymheaviside":
        G = (np.abs(R) >= filterscale).astype(float)
    elif kernel == "symtanh":
        G = -0.5 * (np.tanh(width * (R - filterscale)) + 1) - 0.5 * (np.tanh(-width * (R + filterscale)) - 1)
    elif kernel == "Isymtanh":
        G = -0.5 * (-np.tanh(width * (R - filterscale)) - 1) - 0.5 * (-np.tanh(-width * (R + filterscale)) - 1)
    elif kernel == "boxheaviside":
        n = Nx
        k = np.arange(n)
        k[k > n/2] = k[k > n/2] - n
    
        mask = np.abs(k) < filterscale
        G = np.outer(mask, mask)
    else:
        raise ValueError('ERROR: valid kernels are "tanh", "gaussian", "heaviside", etc.')

    # Apply the filter based on the mode
    filtered_field = np.zeros_like(field)
    if field.ndim == 4:  # Tensor field
        for i in range(field.shape[3]): # iterates over tensor components
            for j in range(field.shape[2]): # iterates over time
                fourier_field = np.fft.fft2(field[:, :, j, i])
                filtered_fourier_field = fourier_field * G
                filtered_field[:, :, j, i] = np.real(np.fft.ifft2(filtered_fourier_field))
    elif field.ndim == 3:  # Single 3D field
        for i in range(field.shape[2]):
            fourier_field = np.fft.fft2(field[:, :, i])
            filtered_fourier_field = fourier_field * G
            filtered_field[:, :, i] = np.real(np.fft.ifft2(filtered_fourier_field))
    else:
        raise ValueError('ERROR: field must have right dimensions')

    return filtered_field


def curl(U, V, dx, dy):
    nx, ny, nt = U.shape
    kx = np.fft.fftfreq(nx, d=dx) * 2 * np.pi
    ky = np.fft.rfftfreq(ny, d=dy) * 2 * np.pi
    kx, ky = np.meshgrid(kx, ky, indexing='ij')
    output = np.zeros((nx, ny, nt))
    for t in range(nt):
        u_FT = np.fft.rfftn(U[:, :, t])
        v_FT = np.fft.rfftn(V[:, :, t])
        i = 1j
        dyu = np.fft.irfftn(i * ky * u_FT, s=(nx, ny))
        dxv = np.fft.irfftn(i * kx * v_FT, s=(nx, ny))
        rhs = dxv - dyu
        rhs_FT = np.fft.rfftn(rhs)
        pressure = np.fft.irfftn(rhs_FT, s=(nx, ny))
        output[:, :, t] = pressure
    return output
    

In [ ]:
# construct observables
filterscale = 256
filterkernel = "boxheaviside"

u = np.stack((U, V), axis=3) # indexed as x,y,t,vector_component
u_bar = fourier_filter(u, 1, filterscale, filterkernel) # MAKE SURE DATA DEALIASED ON MASK WITH CORRESPONDING TO FILTER, ie, circular or rectangular
p_bar = fourier_filter(P, 1, filterscale, filterkernel)

corr_xx = fourier_filter( np.multiply(u[:,:,:,0],u[:,:,:,0]) , 1, filterscale, filterkernel)
corr_xy = fourier_filter( np.multiply(u[:,:,:,0],u[:,:,:,1]) , 1, filterscale, filterkernel)
corr_yy = fourier_filter( np.multiply(u[:,:,:,1],u[:,:,:,1]) , 1, filterscale, filterkernel)

tau_xx = np.multiply(u_bar[:,:,:,0],u_bar[:,:,:,0]) - corr_xx # now here u is indexed as x,y,t instead of matlab's y,x,t
tau_xy = np.multiply(u_bar[:,:,:,0],u_bar[:,:,:,1]) - corr_xy
tau_yy = np.multiply(u_bar[:,:,:,1],u_bar[:,:,:,1]) - corr_yy
del corr_xx
del corr_xy
del corr_yy

print(f"tau_xx: Shape = {tau_xx.shape}, Type = {type(tau_xx)}")
print(f"tau_xy: Shape = {tau_xy.shape}, Type = {type(tau_xy)}")
print(f"tau_yy: Shape = {tau_yy.shape}, Type = {type(tau_yy)}")

b1 = np.stack( ( 0.5*(tau_xx-tau_yy), tau_xy), axis=3) # indexed as x,y,t,vector_component
b2 = np.stack( ( tau_xy, 0.5*(tau_yy-tau_xx)), axis=3)
beta = tau_xx + tau_yy # trace of closure
del tau_xx
del tau_xy
del tau_yy

print(b1.shape)
print(b2.shape)
b = np.stack((b1, b2), axis=4) # traceless component of closure
del b1
del b2
print(b.shape)



In [ ]:
u = np.stack((U, V), axis=3) # indexed as x,y,t,vector_component
bobs = Observable(string='b', rank=2, can_commute_indices = True, antisymmetric = False)
uobs = Observable(string='u', rank=1)
pobs = Observable(string='p', rank=0)
beta = Observable(string='beta', rank=0)
observables = [uobs, pobs, bobs, beta]
data_dict = {'p': p_bar, 'u': u_bar, 'b': b, 'beta': beta}
#data_dict = {'p': P, 'u': u}
# fix random seed
np.random.seed(1)
world_size = np.array(P.shape)
pad = 0
# fix random seed
np.random.seed(1)
dxs = [dx, dy, dt]
# initial setup of dataset
#corr_L = 5
#corr_T = 5
srd = SRDataset(world_size=world_size, data_dict=data_dict, observables=observables, dxs=dxs, 
                irreps=SRDataset.all_rank2_irreps())
                #irreps=(0, 1, 2))
# initialize libraries, domains, and weights
#srd.make_libraries(max_complexity=4, max_observables=3)
srd.make_libraries(max_complexity=4)
#print(srd.libs[0].terms)
dom_width = 32
dom_time = 32 #previously 20 (without interpolation)
srd.make_domains(ndomains=120, domain_size=[dom_width, dom_width, dom_time], pad=pad)
#srd.make_domains(ndomains=10, domain_size=[dom_width, dom_width, dom_time], pad=pad)
srd.make_weights(m=8, qmax=0)
srd.set_LT_scale(L=Lx*Nx/dom_width, T=Lt*Nt/25) # note that this line must go before make_library_matrices
srd.make_library_matrices(debug=False)
from commons_identify_models import *
import copy


In [ ]:
libs = srd.libs
reg_opts_list = []
for irrep in srd.irreps:
    # for regression we now need to construct a Scaler, Initializer, ModelIterator, and Threshold
    scaler = Scaler(sub_inds=None, char_sizes=libs[irrep].col_weights, row_norms=None)
    init = Initializer(method='combinatorial', start_k=2)
    #init = Initializer(method='combinatorial', start_k=9999)
    #init = Initializer(method='power', start_k=10)
    #res = Residual(residual_type='fixed_column', anchor_col=0)
    res = Residual(residual_type='matrix_relative')
    iterator = ModelIterator(max_k=10, backward_forward=True, brute_force=True, max_passes=4)
    thres = Threshold(threshold_type='jump', gamma=1.5, delta=1e-9, n_terms=None)
    #thres = Threshold(threshold_type='information', ic=AIC)
    #thres = Threshold(threshold_type='jump', gamma=1.5, n_terms=3)
    opts = {'scaler': scaler, 'initializer': init, 'residual': res,
            'model_iterator': iterator, 'threshold': thres}
    opts['verbose'] = True
    opts['inhomog'] = False
    opts['inhomog_col'] = None
    reg_opts_list.append(opts)
eqs, lambdas, derived_eqs, excluded_terms = interleave_identify([libs[i] for i in srd.irreps], 
#[opts, opts1, opts2], [libs[i].terms for i in srd.irreps], threshold=1e-4, experimental=True)
#[copy.deepcopy(opts) for i in srd.irreps], [libs[i].terms for i in srd.irreps], threshold=1e-4, experimental=True)
reg_opts_list, threshold=1e-4, experimental=True)